Without fine-tuning

In [2]:
!pip install -q transformers[sentencepiece]

import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Load model and tokenizer
model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set source and target languages
src_lang = "en"
tgt_lang = "ne"
tokenizer.src_lang = src_lang

# Translation function
def translate(text):
    encoded = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.get_lang_id(tgt_lang),
        max_length=128,
        num_beams=4,
        early_stopping=True,
    )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

# Examples
examples = {
    "Easy": "My name is Gyawali.",
    "Medium": "She likes to read books in the evening after finishing her homework.",
    "Hard": "Despite the economic challenges, the government is planning to invest more in renewable energy sources to ensure a sustainable future."
}

# Translate and print results
for level, sentence in examples.items():
    translated = translate(sentence)
    print(f"{level}:\nEnglish: {sentence}\nNepali: {translated}\n")


2025-07-25 19:04:30.104269: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753470270.334081      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753470270.403200      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

Easy:
English: My name is Gyawali.
Nepali: मेरो नाम जियावाली हो ।

Medium:
English: She likes to read books in the evening after finishing her homework.
Nepali: घरबेटीले जहाँ पायो त्यहाँ पकाउन दिँदैन ।

Hard:
English: Despite the economic challenges, the government is planning to invest more in renewable energy sources to ensure a sustainable future.
Nepali: नयाँ संविधानको घोषणा,आन्दोलनको निराशाजनक बैठान,तीनै तहको निर्वाचन पश्चातको देश र मधेसको स्थितिको वास्तविक विश्लेषण हुनु आवश्यक छ ।



In [3]:
!pip install -q transformers datasets sacrebleu

import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import get_scheduler
from tqdm import tqdm
from transformers import DataCollatorForSeq2Seq
from torch.optim import AdamW


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 42.2 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 56.1 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 257.3 kB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.

In [4]:
from datasets import load_dataset

# Load full dataset first
dataset = load_dataset("CohleM/english-to-nepali")

# Display structure
print(dataset)

# Print sample example
print("Example row:")
print(dataset["train"][0])


README.md:   0%|          | 0.00/328 [00:00<?, ?B/s]

(…)-00000-of-00001-ea91b3ffe2804196.parquet:   0%|          | 0.00/28.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/177334 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['en', 'ne'],
        num_rows: 177334
    })
})
Example row:
{'en': 'Technical committees will be attached to each ministry.', 'ne': 'प्रत्येक मन्त्रालय अन्तर्गत शिल्प (टेक्निकल) कमिटीहरु गठन गरिनेछन्'}


In [5]:
# Dataset format: {"en": "...", "ne": "..."}
train_dataset = dataset["train"].train_test_split(test_size=0.1)["train"]
val_dataset = dataset["train"].train_test_split(test_size=0.1)["test"]


In [6]:
model_checkpoint = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_checkpoint)
tokenizer.src_lang = "en"
tokenizer.tgt_lang = "ne"

max_length = 128

def preprocess_function(examples):
    inputs = examples["en"]
    targets = examples["ne"]

    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_length, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=["en", "ne"])
tokenized_val = val_dataset.map(preprocess_function, batched=True, remove_columns=["en", "ne"])


Map:   0%|          | 0/159600 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/17734 [00:00<?, ? examples/s]

In [7]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_checkpoint)

train_dataloader = DataLoader(tokenized_train, shuffle=True, batch_size=8, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_val, batch_size=8, collate_fn=data_collator)


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = M2M100ForConditionalGeneration.from_pretrained(model_checkpoint).to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
num_training_steps = len(train_dataloader) * 1  # 1 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)


In [9]:
model.train()
for epoch in range(1):
    print(f"Epoch {epoch+1}")
    progress_bar = tqdm(train_dataloader)
    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.set_description(f"Loss: {loss.item():.4f}")


Epoch 1


Loss: 0.4467: 100%|██████████| 19950/19950 [3:06:41<00:00,  1.78it/s]  


In [10]:
model.save_pretrained("/kaggle/working/m2m100-en-to-nepali")
tokenizer.save_pretrained("/kaggle/working/m2m100-en-to-nepali")


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('/kaggle/working/m2m100-en-to-nepali/tokenizer_config.json',
 '/kaggle/working/m2m100-en-to-nepali/special_tokens_map.json',
 '/kaggle/working/m2m100-en-to-nepali/vocab.json',
 '/kaggle/working/m2m100-en-to-nepali/sentencepiece.bpe.model',
 '/kaggle/working/m2m100-en-to-nepali/added_tokens.json')

In [11]:
!zip -r /kaggle/working/m2m100-en-to-nepali.zip /kaggle/working/m2m100-en-to-nepali


  adding: kaggle/working/m2m100-en-to-nepali/ (stored 0%)
  adding: kaggle/working/m2m100-en-to-nepali/added_tokens.json (deflated 76%)
  adding: kaggle/working/m2m100-en-to-nepali/tokenizer_config.json (deflated 94%)
  adding: kaggle/working/m2m100-en-to-nepali/special_tokens_map.json (deflated 79%)
  adding: kaggle/working/m2m100-en-to-nepali/vocab.json (deflated 71%)
  adding: kaggle/working/m2m100-en-to-nepali/sentencepiece.bpe.model (deflated 50%)
  adding: kaggle/working/m2m100-en-to-nepali/generation_config.json (deflated 38%)
  adding: kaggle/working/m2m100-en-to-nepali/config.json (deflated 57%)
  adding: kaggle/working/m2m100-en-to-nepali/model.safetensors (deflated 7%)


In [12]:
!ls /kaggle/working/m2m100-en-to-nepali

added_tokens.json	model.safetensors	 tokenizer_config.json
config.json		sentencepiece.bpe.model  vocab.json
generation_config.json	special_tokens_map.json


In [13]:
def translate(text):
    tokenizer.src_lang = "en"
    encoded = tokenizer(text, return_tensors="pt", padding=True).to(device)
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("ne"))
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

# Example
print(translate("I'm not lazy."))


म व्यस्त छैन ।


In [14]:
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load saved model and tokenizer
model_path = "/kaggle/working/m2m100-en-to-nepali"  # folder where model & tokenizer were saved
tokenizer = M2M100Tokenizer.from_pretrained(model_path)
model = M2M100ForConditionalGeneration.from_pretrained(model_path).to(device)

# Translate function
def translate(text):
    tokenizer.src_lang = "en"
    encoded = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.get_lang_id("ne"),
        max_length=128,
        num_beams=4,
        early_stopping=True,
    )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

# Example usage
print(translate("My name is Navin."))


मेरो नाम नाभिन हो।


In [15]:
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load saved model and tokenizer
model_path = "/kaggle/working/m2m100-en-to-nepali"
tokenizer = M2M100Tokenizer.from_pretrained(model_path)
model = M2M100ForConditionalGeneration.from_pretrained(model_path).to(device)

# Translate function
def translate(text):
    tokenizer.src_lang = "en"
    encoded = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.get_lang_id("ne"),
        max_length=128,
        num_beams=4,
        early_stopping=True,
    )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

# Test sentences
examples = {
    "Easy": "My name is Gyawali.",
    "Medium": "She likes to read books in the evening after finishing her homework.",
    "Hard": "Despite the economic challenges, the government is planning to invest more in renewable energy sources to ensure a sustainable future."
}
# Run translations
for level, sentence in examples.items():
    translated = translate(sentence)
    print(f"{level}:\nEnglish: {sentence}\nNepali: {translated}\n")


Easy:
English: My name is Gyawali.
Nepali: मेरो नाम गिवाली हो।

Medium:
English: She likes to read books in the evening after finishing her homework.
Nepali: उनी आफ्नो गृह कार्य समाप्त भएपछि साँझमा किताबहरू पढ्न मन पराउँछन्।

Hard:
English: Despite the economic challenges, the government is planning to invest more in renewable energy sources to ensure a sustainable future.
Nepali: आर्थिक चुनौतिका बाबजुद सरकारले दिगो भविष्यलाई सुनिश्चित गर्न नवीकरणीय ऊर्जाको स्रोतमा बढी लगानी गर्ने योजना रहेको छ।



In [16]:
import time

# Loop through examples and measure translation time
for level, sentence in examples.items():
    start_time = time.time()
    translated = translate(sentence)
    end_time = time.time()
    
    elapsed_time = end_time - start_time
    
    print(f"{level}:\nEnglish: {sentence}\nNepali: {translated}")
    print(f"Time taken: {elapsed_time:.4f} seconds\n")


Easy:
English: My name is Gyawali.
Nepali: मेरो नाम गिवाली हो।
Time taken: 0.1395 seconds

Medium:
English: She likes to read books in the evening after finishing her homework.
Nepali: उनी आफ्नो गृह कार्य समाप्त भएपछि साँझमा किताबहरू पढ्न मन पराउँछन्।
Time taken: 0.2492 seconds

Hard:
English: Despite the economic challenges, the government is planning to invest more in renewable energy sources to ensure a sustainable future.
Nepali: आर्थिक चुनौतिका बाबजुद सरकारले दिगो भविष्यलाई सुनिश्चित गर्न नवीकरणीय ऊर्जाको स्रोतमा बढी लगानी गर्ने योजना रहेको छ।
Time taken: 0.4119 seconds



In [17]:
!pip install sacrebleu bert-score nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 24.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nv

In [21]:
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import sacrebleu
import bert_score

# Download punkt tokenizer for nltk BLEU (if needed)
nltk.download('punkt')

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and tokenizer
model_path = "/kaggle/working/m2m100-en-to-nepali"
tokenizer = M2M100Tokenizer.from_pretrained(model_path)
model = M2M100ForConditionalGeneration.from_pretrained(model_path).to(device)

# Translate function
def translate(text):
    tokenizer.src_lang = "en"
    encoded = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    generated_tokens = model.generate(
        **encoded,
        forced_bos_token_id=tokenizer.get_lang_id("ne"),
        max_length=128,
        num_beams=4,
        early_stopping=True,
    )
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

# Test examples with reference Nepali translations (ground truth)
examples = {
    "Easy": {
        "en": "My name is Gyawali.",
        "ne": "मेरो नाम ज्ञवली हो।"
    },
    "Medium": {
        "en": "he likes to read books in the evening after finishing her homework.",
        "ne": "उ गृहकार्य सकेपछि साँझ किताब पढ्न मन पराउँछ।"
    },
    "Hard": {
        "en": "Despite the economic challenges, the government is planning to invest more in renewable energy sources to ensure a sustainable future.",
        "ne": "आर्थिक चुनौतीहरूका बाबजुद, सरकारले दिगो भविष्य सुनिश्चित गर्न नवीकरणीय ऊर्जा स्रोतहरूमा थप लगानी गर्ने योजना बनाइरहेको छ।"
    }
}


# Evaluation loop
for level, data in examples.items():
    src = data["en"]
    ref = data["ne"]
    
    pred = translate(src)
    
    # BLEU score (sentence-level with smoothing)
    smoothie = SmoothingFunction().method4
    ref_tokens = [nltk.word_tokenize(ref)]
    pred_tokens = nltk.word_tokenize(pred)
    bleu_score = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smoothie)
    
    # SacreBLEU score (alternative BLEU)
    sacre_bleu = sacrebleu.sentence_bleu(pred, [ref]).score
    
    
    # BERTScore (semantic similarity)
    P, R, F1 = bert_score.score([pred], [ref], lang="ne", verbose=False)
    bert_f1 = F1[0].item()
    
    print(f"--- {level} ---")
    print(f"English: {src}")
    print(f"Reference Nepali: {ref}")
    print(f"Predicted Nepali: {pred}\n")
    
    print(f"BLEU (nltk)     : {bleu_score:.4f}")
    print(f"SacreBLEU       : {sacre_bleu:.2f}")
    print(f"BERTScore F1    : {bert_f1:.4f}")
    print()


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


--- Easy ---
English: My name is Gyawali.
Reference Nepali: मेरो नाम ज्ञवली हो।
Predicted Nepali: मेरो नाम गिवाली हो।

BLEU (nltk)     : 0.1862
SacreBLEU       : 35.36
BERTScore F1    : 0.8944

--- Medium ---
English: he likes to read books in the evening after finishing her homework.
Reference Nepali: उ गृहकार्य सकेपछि साँझ किताब पढ्न मन पराउँछ।
Predicted Nepali: उनी आफ्नो गृह कार्य समाप्त भएपछि साँझमा किताब पढ्न मन पराउँछन्।

BLEU (nltk)     : 0.1161
SacreBLEU       : 13.95
BERTScore F1    : 0.8889

--- Hard ---
English: Despite the economic challenges, the government is planning to invest more in renewable energy sources to ensure a sustainable future.
Reference Nepali: आर्थिक चुनौतीहरूका बाबजुद, सरकारले दिगो भविष्य सुनिश्चित गर्न नवीकरणीय ऊर्जा स्रोतहरूमा थप लगानी गर्ने योजना बनाइरहेको छ।
Predicted Nepali: आर्थिक चुनौतिका बाबजुद सरकारले दिगो भविष्यलाई सुनिश्चित गर्न नवीकरणीय ऊर्जाको स्रोतमा बढी लगानी गर्ने योजना रहेको छ।

BLEU (nltk)     : 0.1441
SacreBLEU       : 16.61
BERTScore F